Hypothesis: Participants who have an average of 3 locations per week will have higher stress algorithms than people who use an average of 4 or more locations per week. 

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt  # To visualize
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
SIX_MONTH_DAILY_AM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily AM /7103000 (3).csv"
SIX_MONTH_DAILY_PM_PATH_1 =  "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily PM /7106000 (2).csv"
SIX_MONTH_DAILY_PM_PATH_2 = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily PM /7592000.csv"
SIX_MONTH_DAILY_END_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily END/7591000.csv"
SIX_MONTH_DAILY_END_PATH2 = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily END/7108000 (2).csv"
SIX_MONTH_FRIDAY_AM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday AM /7105000 (2).csv"
SIX_MONTH_FRIDAY_PM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday PM/7107000 (2).csv"
SIX_MONTH_FRIDAY_END_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday END/7109000 (2).csv"


In [4]:
SIX_MONTH_DAILY_AM = pd.read_csv(SIX_MONTH_DAILY_AM_PATH)
SIX_MONTH_DAILY_PM = pd.concat([pd.read_csv(SIX_MONTH_DAILY_PM_PATH_1, encoding='cp1252'), pd.read_csv(SIX_MONTH_DAILY_PM_PATH_2)])
SIX_MONTH_DAILY_END = pd.concat([pd.read_csv(SIX_MONTH_DAILY_END_PATH), pd.read_csv(SIX_MONTH_DAILY_END_PATH2)])
SIX_MONTH_FRIDAY_AM = pd.read_csv(SIX_MONTH_FRIDAY_AM_PATH)
SIX_MONTH_FRIDAY_PM = pd.read_csv(SIX_MONTH_FRIDAY_PM_PATH, encoding='cp1252')
SIX_MONTH_FRIDAY_END = pd.read_csv(SIX_MONTH_FRIDAY_END_PATH)

In [5]:
import datetime as dt
def get_week():
  begin = dt.datetime(2022, 4, 24)
  end = dt.datetime.now()
  week_dict = {}
  week_num = 0
  while begin < end:
    curBeg = begin
    curEnd = begin + dt.timedelta(days = 7)
    begin = curEnd
    week_dict[curEnd] = week_num
    week_num += 1
  return week_dict

week_dict = get_week()

In [6]:
def split_date_time(DATA):
  DATA[["date", "exact_time"]] = DATA["local_time"].str.split(" ", expand=True)
  DATA = DATA.drop(columns = ['local_time'])
  DATA[["Year", "Month", "Day"]] = DATA["date"].str.split("-", expand=True)
  DATA = DATA.drop(columns = ["date"])
  return DATA

SIX_MONTH_DAILY_AM = split_date_time(SIX_MONTH_DAILY_AM)
SIX_MONTH_DAILY_PM = split_date_time(SIX_MONTH_DAILY_PM)
SIX_MONTH_DAILY_END = split_date_time(SIX_MONTH_DAILY_END)
SIX_MONTH_FRIDAY_AM = split_date_time(SIX_MONTH_FRIDAY_AM)
SIX_MONTH_FRIDAY_PM = split_date_time(SIX_MONTH_FRIDAY_PM)
SIX_MONTH_FRIDAY_END = split_date_time(SIX_MONTH_FRIDAY_END)

In [7]:
SIX_MONTH_DAILY_MERGE_1 = pd.merge(SIX_MONTH_DAILY_AM, SIX_MONTH_DAILY_PM, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_DAILY = pd.merge(SIX_MONTH_DAILY_MERGE_1, SIX_MONTH_DAILY_END, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_FRIDAY_MERGE_1 = pd.merge(SIX_MONTH_FRIDAY_AM, SIX_MONTH_FRIDAY_PM, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_FRIDAY = pd.merge(SIX_MONTH_FRIDAY_MERGE_1, SIX_MONTH_FRIDAY_END, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])

In [ ]:
SIX_MONTH_DAILY_LOCATION = SIX_MONTH_DAILY[['mbl_cod', 'LOCATION_AM', 'LOCATION_PM', 'LOCATION_END', 'Year', 'Month', 'Day']]
SIX_MONTH_FRIDAY_LOCATION = SIX_MONTH_FRIDAY[['mbl_cod', 'LOCATION_AM', 'LOCATION_PM', 'LOCATION_END', 'Year', 'Month', 'Day']]
SIX_MONTH_DAILY_LOCATION['Week Number'] = None
SIX_MONTH_FRIDAY_LOCATION['Week Number'] = None

In [9]:
def week_num(row):
  datetime = dt.datetime(int(row[4]), int(row[5]), int(row[6]))
  day_of_week = datetime.weekday()
  offset = 6 - day_of_week
  new_date = datetime + dt.timedelta(days = offset)
  row[-1] = week_dict[new_date]
  return row


SIX_MONTH_DAILY_LOCATION = SIX_MONTH_DAILY_LOCATION.apply(week_num, axis = 1)
SIX_MONTH_FRIDAY_LOCATION = SIX_MONTH_FRIDAY_LOCATION.apply(week_num, axis = 1)

In [10]:
SIX_MONTH_LOCATION = pd.merge(SIX_MONTH_DAILY_LOCATION, SIX_MONTH_FRIDAY_LOCATION, on = ['mbl_cod', 'Week Number'], how = "left")

In [11]:
from collections import Counter
grouped = SIX_MONTH_LOCATION.groupby(['mbl_cod', 'Week Number'])

new_df = pd.DataFrame()
mbl_cod = []
week_number = []
distinct = []
for key, item in grouped:
    a = grouped.get_group(key)
    a = a.dropna()
    if a.shape[0] == 0:
      continue
    mbl_cod.append(list(a['mbl_cod'])[0])
    am_data = list(a['LOCATION_AM_x'])
    pm_data = list(a['LOCATION_PM_x'])
    end_data = list(a['LOCATION_END_x'])
    friday_am_data = list(a['LOCATION_AM_y'])
    friday_pm_data = list(a['LOCATION_PM_y'])
    friday_end_data = list(a['LOCATION_END_y'])
    week_number.append(list(a['Week Number'])[0])
    all_data = am_data + pm_data + end_data + friday_am_data + friday_pm_data + friday_end_data
    all_data = [int(x) for x in all_data if isinstance(x, float) or isinstance(x, int)]
    distinct_data = len(Counter(all_data).keys())
    distinct.append(distinct_data)

In [12]:
new_df['mbl_cod'] = mbl_cod
new_df['Week Number'] = week_number
new_df['Number of Locations'] = distinct
new_df.head()

,mbl_cod,Week Number,Number of Locations
0,11822993,2,5
1,11822993,10,3
2,11822993,12,4
3,11822993,13,3
4,17180706,4,3


In [ ]:
import os
dir_path_garmin = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Garmin Data"
print(os.listdir(dir_path_garmin))
all_data = None
for file in os.listdir(dir_path_garmin):
  mbr_col = file.split(".")[1].split(" ")[0]
  full_path = os.path.join(dir_path_garmin, file)
  df = pd.read_csv(full_path)
  df['mbr_col'] = mbr_col
  df = df[df['data_type'] == "stress"]
  df = df[df['val'] != 0]
  if isinstance(all_data, pd.core.frame.DataFrame):
    all_data = pd.concat([all_data, df])
  else:
    all_data = df
    


In [ ]:
all_data.to_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/garmin_combined.csv")

In [ ]:
all_data_dropped = all_data[['mbr_col', 'dte_tme', 'val']]

In [ ]:
all_data_dropped['year'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).year
all_data_dropped['month'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).month
all_data_dropped['day'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).day
all_data_dropped = all_data_dropped.drop(columns = ['dte_tme'])

In [ ]:
def week_num(row):
  datetime = dt.datetime(int(row[2]), int(row[3]), int(row[4]))
  day_of_week = datetime.weekday()
  offset = 6 - day_of_week
  new_date = datetime + dt.timedelta(days = offset)
  row[-1] = week_dict[new_date]
  return row


all_data_dropped['Week Number'] = None
all_data_dropped = all_data_dropped.apply(week_num, axis = 1)

In [ ]:
grouped_data = pd.read_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/grouped_garmin.csv")
grouped_data = grouped_data.rename(columns = {"mbr_col" : "mbl_cod", "val" : "min_number", "val.1" : "max_number", "val.2": "mean_number"})

In [ ]:
final_df = pd.merge(new_df, grouped_data, on = ['mbl_cod', 'Week Number'])
final_df_analysis = final_df[['Number of Locations', 'min_number', 'max_number', 'mean_number']]

In [ ]:
final_df_analysis['GreaterFour'] = None

def greater_four(row):
  row[-1] = 0 if row[0] < 4 else 1
  return row

final = final_df_analysis.apply(greater_four, axis = 1)

In [ ]:
final.to_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/full_df.csv")

In [ ]:
get_vals = list(final['mean_number'])
get_vals = [str(x) for x in get_vals]
get_vals = [float(x) for x in get_vals]
final['mean_number'] = get_vals

get_vals = list(final['max_number'])
get_vals = [str(x) for x in get_vals]
get_vals = [float(x) for x in get_vals]
final['max_number'] = get_vals



In [ ]:
grouped = final.groupby('GreaterFour').mean_number.mean()

In [ ]:
import seaborn as sb  
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot as plt
from scipy.stats import norm, uniform
import numpy

In [ ]:
import plotly.express as px

fig = px.box(final, x="GreaterFour", y="mean_number")
fig.show()

In [ ]:
df1 = final[final['GreaterFour'] == 0]
df2 = final[final['GreaterFour'] == 1]

In [ ]:
list_fourmore = list(df1['mean_number'])
list_threeless = list(df2['mean_number'])

In [ ]:
from scipy.stats import f_oneway
stat, p = f_oneway(list_fourmore, list_threeless)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('We can not reject the null hypothesis')
else:
	print('We reject the hypothesis')

In [ ]:
from scipy.stats import ttest_rel
from scipy import stats

stats.ttest_ind(list_fourmore, list_threeless, equal_var = False)

In [ ]:
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
import pandas as pd

def welch_ttest(x1, x2,alternative):
    
    n1 = len(x1)
    n2 = len(x2)
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    
    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)
    
    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2
    
    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))
    
    # two side t-test
    p = 2 * t.cdf(-abs(tstat), df)
    
    # upper and lower bounds
    lb = delta - t.ppf(0.975,df)*pooled_se 
    ub = delta + t.ppf(0.975,df)*pooled_se
  
    return pd.DataFrame(np.array([tstat,df,p,delta,lb,ub]).reshape(1,-1),
                         columns=['T statistic','df','pvalue 2 sided','Difference in mean','lb','ub'])

In [ ]:
print(welch_ttest(list_threeless,list_fourmore,"equal"))

PER PERSON FINDINGS

In [13]:
grouped_data = pd.read_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/BU Sargent Well Being/Team 3/Deliverable 3/grouped_garmin.csv")
grouped_data = grouped_data.rename(columns = {"mbr_col": "mbl_cod", "val" : "min", "val.1" : "max", "val.2" : "mean"})
grouped_data = grouped_data.iloc[1:][:]
grouped_data = grouped_data.drop(columns = ["Unnamed: 0"])

In [14]:
full_df = pd.read_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/BU Sargent Well Being/Team 3/Deliverable 3/full_df.csv")

In [15]:
merged = pd.merge(grouped_data, new_df, on = ["mbl_cod", "Week Number"])

In [16]:
unique_people = merged.mbl_cod.unique()
df_dictionary = {x : None for x in unique_people}
for key in df_dictionary:
  df = merged[merged['mbl_cod'] == key]
  number_locations = df["Number of Locations"].values
  stress = df['mean'].values
  fourmore = []
  threeless = []
  for i in range(len(number_locations)):
    if number_locations[i] >= 4:
      fourmore.append(stress[i])
    if number_locations[i] <= 3:
      threeless.append(stress[i])
  
  df_dictionary[key] = [threeless, fourmore]

In [123]:
from scipy import stats
no_data = []
statistically_sig = []
non_statisticall_sig = []
for key in df_dictionary:
  data = df_dictionary[key]
  fourmore = data[1]
  threeless = data[0]
  fourmore = [float(x) for x in fourmore]
  threeless = [float(x) for x in threeless]
  if len(fourmore) == 0 or len(threeless) == 0:
    no_data.append(key)
    continue
  else:
    stat, p = stats.ttest_ind(threeless, fourmore, equal_var = False)
    if p > 0.05:
      non_statisticall_sig.append(key)
    else:
      statistically_sig.append(key)


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [124]:
statistically_sig

[11822993.0,
 22541511.0,
 33075391.0,
 37720972.0,
 58601340.0,
 60404747.0,
 61307863.0,
 64811087.0,
 77253909.0,
 81862952.0]

In [125]:
non_statisticall_sig

[27148444.0,
 32455277.0,
 32937810.0,
 34865333.0,
 35549180.0,
 38656882.0,
 38876664.0,
 51755925.0,
 56954906.0,
 57026233.0,
 66958688.0,
 66999191.0,
 69497234.0,
 79316883.0,
 86548395.0]

In [22]:
SIX_MONTH_DAILY_AM = pd.read_csv(SIX_MONTH_DAILY_AM_PATH)
SIX_MONTH_DAILY_PM = pd.concat([pd.read_csv(SIX_MONTH_DAILY_PM_PATH_1, encoding='cp1252'), pd.read_csv(SIX_MONTH_DAILY_PM_PATH_2)])
SIX_MONTH_DAILY_END = pd.concat([pd.read_csv(SIX_MONTH_DAILY_END_PATH), pd.read_csv(SIX_MONTH_DAILY_END_PATH2)])
SIX_MONTH_FRIDAY_AM = pd.read_csv(SIX_MONTH_FRIDAY_AM_PATH)
SIX_MONTH_FRIDAY_PM = pd.read_csv(SIX_MONTH_FRIDAY_PM_PATH, encoding='cp1252')
SIX_MONTH_FRIDAY_END = pd.read_csv(SIX_MONTH_FRIDAY_END_PATH)

In [127]:
SIX_MONTH_DAILY_AM_SIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_DAILY_AM_NOTSIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(non_statisticall_sig)]

SIX_MONTH_DAILY_PM_SIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_DAILY_PM_NOTSIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(non_statisticall_sig)]

SIX_MONTH_DAILY_END_SIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_DAILY_END_NOTSIG = SIX_MONTH_DAILY_AM[SIX_MONTH_DAILY_AM['mbl_cod'].isin(non_statisticall_sig)]

SIX_MONTH_FRIDAY_AM_SIG = SIX_MONTH_FRIDAY_AM[SIX_MONTH_FRIDAY_AM['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_FRIDAY_AM_NOTSIG = SIX_MONTH_FRIDAY_AM[SIX_MONTH_FRIDAY_AM['mbl_cod'].isin(non_statisticall_sig)]

SIX_MONTH_FRIDAY_PM_SIG = SIX_MONTH_FRIDAY_PM[SIX_MONTH_FRIDAY_PM['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_FRIDAY_PM_NOTSIG = SIX_MONTH_FRIDAY_PM[SIX_MONTH_FRIDAY_PM['mbl_cod'].isin(non_statisticall_sig)]

SIX_MONTH_FRIDAY_END_SIG = SIX_MONTH_FRIDAY_END[SIX_MONTH_FRIDAY_END['mbl_cod'].isin(statistically_sig)]
SIX_MONTH_FRIDAY_END_NOTSIG = SIX_MONTH_FRIDAY_END[SIX_MONTH_FRIDAY_END['mbl_cod'].isin(non_statisticall_sig)]




In [128]:
def get_all_discomforts(df1, df2, df3, df4):
  discomfort = []
  discomfort += list(df1['DISCOMFORT_SLIDER'])
  discomfort += list(df2['DISCOMFORT_SLIDER'])
  discomfort += list(df3['DISCOMFORT_SLIDER'])
  discomfort += list(df4['DISCOMFORT_SLIDER'])
  return discomfort

In [129]:
STAT_SIG_DISCOMFORT = get_all_discomforts(SIX_MONTH_DAILY_AM_SIG, SIX_MONTH_DAILY_PM_SIG, SIX_MONTH_DAILY_END_SIG, SIX_MONTH_FRIDAY_AM_SIG)
NOTSTAT_SIG_DISCOMFORT = get_all_discomforts(SIX_MONTH_DAILY_AM_NOTSIG, SIX_MONTH_DAILY_PM_NOTSIG, SIX_MONTH_DAILY_END_NOTSIG, SIX_MONTH_FRIDAY_AM_NOTSIG)

In [130]:
AVG_STAT_SIG = sum(STAT_SIG_DISCOMFORT) / len(STAT_SIG_DISCOMFORT)
AVG_STAT_NOTSIG = sum(NOTSTAT_SIG_DISCOMFORT) / len(NOTSTAT_SIG_DISCOMFORT)

In [131]:
AVG_STAT_SIG

2.1270373921380634

In [132]:
AVG_STAT_NOTSIG

1.3825136612021858

In [133]:
from scipy.stats import f_oneway
stat, p = f_oneway(STAT_SIG_DISCOMFORT, NOTSTAT_SIG_DISCOMFORT)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('We can not reject the null hypothesis')
else:
	print('We reject the hypothesis')

stat=498.353, p=0.000
We reject the hypothesis


IS STATISTICALLY SIGINIFICANT

In [134]:
SIX_MONTH_FRIDAY_AM_SIG.columns

Index(['mbl_cod', 'rsp_id', 'ts', 'local_time', 'LOCATION_AM',
       'DISCOMFORT_SLIDER', 'LIFE_SATISFACTION', 'HAPPINESS',
       'PHYSICAL_HEALTH', 'MENTAL_HEALTH', 'WORTHWHILE', 'PURPOSE',
       'PROMOTE_GOOD', 'DELAYED_HAPPINESS', 'CONTENT_RELATIONSHIPS',
       'SATISFYING_RELATIONSHIPS', 'LIVING_EXPENSES', 'FOOD_HOUSING', 'STRESS',
       'PULSE_OX', 'HEART_RATE', 'RESPIRATION', 'BODY_BATTERY', 'STEPS',
       'CALORIES', 'FLOORS', 'INTENSITY_MINUTES', 'LONGITUDE', 'LATITUDE',
       'AVG_AMP', 'VOX_ACTV'],
      dtype='object')

In [79]:
SIX_MONTH_FRIDAY_PM_NOTSIG.columns

Index(['mbl_cod', 'rsp_id', 'ts', 'local_time', 'LOCATION_PM',
       'HOMUNCULUS_INTRO', 'HOMUNCULUS', 'OTHER_DISCOMFORT', 'TRUST_1',
       'TRUST_2', 'TRUST_3', 'TRUST_4', 'TRUST_5', 'FLEXIBILITY_6',
       'FLEXIBILITY_7', 'FLEXIBILITY_8', 'FLEXIBILITY_9', 'WORK_LIFE_10',
       'WORK_LIFE_11', 'WORK_LIFE_12', 'WORK_LIFE_13', 'WORK_LIFE_14',
       'WORK_LIFE_15', 'PRODUCTIVITY_16', 'PRODUCTIVITY_17', 'PRODUCTIVITY_18',
       'PRODUCTIVITY_19', 'PRODUCTIVITY_20', 'PRODUCTIVITY_21', 'STRESS',
       'PULSE_OX', 'HEART_RATE', 'RESPIRATION', 'BODY_BATTERY', 'STEPS',
       'CALORIES', 'FLOORS', 'INTENSITY_MINUTES', 'LONGITUDE', 'LATITUDE',
       'AVG_AMP', 'VOX_ACTV'],
      dtype='object')

In [80]:
dictionary_am_columns = {'LIFE_SATISFACTION': None, 'HAPPINESS':None,
       'PHYSICAL_HEALTH':None, 'MENTAL_HEALTH':None, 'WORTHWHILE':None, 'PURPOSE':None,
       'PROMOTE_GOOD':None, 'DELAYED_HAPPINESS':None, 'CONTENT_RELATIONSHIPS':None,
       'SATISFYING_RELATIONSHIPS':None, 'LIVING_EXPENSES':None, 'FOOD_HOUSING':None}

In [81]:
dictionary_pm_columns = {'TRUST_1': None,
       'TRUST_2':None, 'TRUST_3':None, 'TRUST_4':None, 'TRUST_5':None, 'FLEXIBILITY_6':None,
       'FLEXIBILITY_7':None, 'FLEXIBILITY_8':None, 'FLEXIBILITY_9':None, 'WORK_LIFE_10':None,
       'WORK_LIFE_11':None, 'WORK_LIFE_12':None, 'WORK_LIFE_13':None, 'WORK_LIFE_14':None,
       'WORK_LIFE_15':None, 'PRODUCTIVITY_16':None, 'PRODUCTIVITY_17':None, 'PRODUCTIVITY_18':None,
       'PRODUCTIVITY_19':None, 'PRODUCTIVITY_20':None, 'PRODUCTIVITY_21':None}

In [135]:
def extract_dictionary(dictionary, df_sig, df_notsig):
  for key in dictionary:
    sig = list(df_sig[key].dropna())
    not_sig = list(df_notsig[key].dropna())
    dictionary[key] = [sig, not_sig]
  return dictionary

In [136]:
new_dictionary_am = extract_dictionary(dictionary_am_columns, SIX_MONTH_FRIDAY_AM_SIG, SIX_MONTH_FRIDAY_AM_NOTSIG)
new_dictionary_pm = extract_dictionary(dictionary_pm_columns, SIX_MONTH_FRIDAY_PM_SIG, SIX_MONTH_FRIDAY_PM_NOTSIG)

In [137]:
def return_sig(dictionary):
  statistically_significant = []
  non_statistically_significant = []
  for key in dictionary:
    data = dictionary[key]
    sig = data[0]
    notsig = data[1]
    avg_sig = sum(sig) / len(sig)
    avg_notsig = sum(notsig) / len(notsig)
    stat, p = stats.ttest_ind(sig, notsig, equal_var = False)
    if p > 0.05:
      non_statistically_significant.append([key, avg_sig > avg_notsig])
    else:
      statistically_significant.append([key, avg_sig > avg_notsig])
  return statistically_significant, non_statistically_significant

In [138]:
stat_sig_am, stat_notsig_am = return_sig(new_dictionary_am)
stat_sig_pm, stat_notsig_pm = return_sig(new_dictionary_pm)

In [142]:
stat_sig_am

[['LIFE_SATISFACTION', False],
 ['HAPPINESS', False],
 ['PHYSICAL_HEALTH', False],
 ['MENTAL_HEALTH', False],
 ['WORTHWHILE', False],
 ['PURPOSE', False],
 ['CONTENT_RELATIONSHIPS', False],
 ['LIVING_EXPENSES', False],
 ['FOOD_HOUSING', False]]

In [140]:
stat_sig_pm

[['TRUST_2', True],
 ['TRUST_3', False],
 ['TRUST_4', False],
 ['TRUST_5', False],
 ['FLEXIBILITY_6', True],
 ['FLEXIBILITY_7', False],
 ['FLEXIBILITY_8', False],
 ['FLEXIBILITY_9', False],
 ['WORK_LIFE_10', True],
 ['WORK_LIFE_11', True],
 ['WORK_LIFE_12', True],
 ['WORK_LIFE_13', True],
 ['WORK_LIFE_14', True],
 ['PRODUCTIVITY_20', True],
 ['PRODUCTIVITY_21', True]]